## Dataset -> CSV resumido

In [1]:
import pandas as pd
import re
import gzip

In [ ]:
pattern_refs = re.compile(r'href="/wiki/([^"#:]*)"')
pattern_title = re.compile(r"[?&]title=([^&]+)")
pattern_categories = re.compile(r'\/wiki\/Category:([^"#:]*)')

In [65]:
del_columns = [
    "annotations", "document_tokens", "long_answer_candidates",
    "question_text", "question_tokens", "example_id", "document_html"
]

output_file = "simplified_nq_processed.csv"

In [ ]:
chunk_size = 2000

for idx in range(50):  
    input_file = f"v1.0/train/nq-train-{idx:02d}.jsonl.gz"   # entrada
    output_file = f"csv/nq-train-{idx:02d}.csv"       # saída
    write_header = True  

    with gzip.open(input_file, "rt", encoding="utf-8") as f:
        batch = []
        for i, line in enumerate(f, 1):
            batch.append(pd.read_json(pd.io.common.StringIO(line), lines=True))

            if i % chunk_size == 0:
                df = pd.concat(batch, ignore_index=True)
                batch = []

                # processa colunas
                df["refs"] = df["document_html"].str.findall(pattern_refs)
                df["title_encode"] = df["document_url"].str.extract(pattern_title)

                for col in del_columns:
                    if col in df.columns:
                        del df[col]

                # salva no CSV (um por arquivo)
                df.to_csv(output_file, mode="a", index=False, header=write_header)
                write_header = False

        # processa o resto
        if batch:
            df = pd.concat(batch, ignore_index=True)

            df["refs"] = df["document_html"].str.findall(pattern_refs)
            df["title_encode"] = df["document_url"].str.extract(pattern_title)

            for col in del_columns:
                if col in df.columns:
                    del df[col]

            df.to_csv(output_file, mode="a", index=False, header=write_header)
#422m 19.9s

In [67]:
df["num_refs"] = df["refs"].apply(len)
media_refs = df["num_refs"].mean()
print("Média de refs por linha:", media_refs)

Média de refs por linha: 382.53333333333336


In [59]:
df["self_ref"] = df.apply(lambda row: row["title_encode"] in row["refs"], axis=1)
# Porcentagem de linhas que se auto referenciam
percent_self_ref = df["self_ref"].mean() * 100
print(df)
print(f"Porcentagem de auto referência: {percent_self_ref:.2f}%")

                                 document_title  \
0            Orange Is the New Black (season 5)   
1                            Harlem Renaissance   
2                                     Pemberley   
3                                History of IBM   
4       Graveyard Shift (SpongeBob SquarePants)   
...                                         ...   
1825  Natasha, Pierre & The Great Comet of 1812   
1826                         Sarbanes–Oxley Act   
1827                                 Divergence   
1828                             Cardiac output   
1829                    Smoke Gets in Your Eyes   

                                           document_url  \
0     https://en.wikipedia.org//w/index.php?title=Or...   
1     https://en.wikipedia.org//w/index.php?title=Ha...   
2     https://en.wikipedia.org//w/index.php?title=Pe...   
3     https://en.wikipedia.org//w/index.php?title=Hi...   
4     https://en.wikipedia.org//w/index.php?title=Gr...   
...                              

In [61]:
from collections import Counter
def count_repeated(lst):
    counts = Counter(lst)
    return sum(1 for v in counts.values() if v > 1)  # número de elementos repetidos
    # se quiser contar **quantas vezes se repetem**:
    # return sum(v-1 for v in counts.values() if v > 1)

df['repetidos'] = df['refs'].apply(count_repeated)
print(df)

                                 document_title  \
0            Orange Is the New Black (season 5)   
1                            Harlem Renaissance   
2                                     Pemberley   
3                                History of IBM   
4       Graveyard Shift (SpongeBob SquarePants)   
...                                         ...   
1825  Natasha, Pierre & The Great Comet of 1812   
1826                         Sarbanes–Oxley Act   
1827                                 Divergence   
1828                             Cardiac output   
1829                    Smoke Gets in Your Eyes   

                                           document_url  \
0     https://en.wikipedia.org//w/index.php?title=Or...   
1     https://en.wikipedia.org//w/index.php?title=Ha...   
2     https://en.wikipedia.org//w/index.php?title=Pe...   
3     https://en.wikipedia.org//w/index.php?title=Hi...   
4     https://en.wikipedia.org//w/index.php?title=Gr...   
...                              

## CSV -> KG (Neo4J)

In [62]:
df

,document_title,document_url,refs,title_encode,num_refs,self_ref,repetidos
0,Orange Is the New Black (season 5),https://en.wikipedia.org//w/index.php?title=Or...,"[Taylor_Schilling, Natasha_Lyonne, Uzo_Aduba, ...",Orange_Is_the_New_Black_(season_5),151,True,35
1,Harlem Renaissance,https://en.wikipedia.org//w/index.php?title=Ha...,"[New_York_Renaissance, Harlem, Alain_Locke, Gr...",Harlem_Renaissance,401,True,29
2,Pemberley,https://en.wikipedia.org//w/index.php?title=Pe...,"[Fitzwilliam_Darcy, Jane_Austen, Pride_and_Pre...",Pemberley,85,True,16
3,History of IBM,https://en.wikipedia.org//w/index.php?title=Hi...,"[IBM, Armonk,_New_York, Unit_record_equipment,...",History_of_IBM,670,True,103
4,Graveyard Shift (SpongeBob SquarePants),https://en.wikipedia.org//w/index.php?title=Gr...,"[SpongeBob_SquarePants, SpongeBob_SquarePants_...",Graveyard_Shift_(SpongeBob_SquarePants),125,True,19
...,...,...,...,...,...,...,...
1825,"Natasha, Pierre & The Great Comet of 1812",https://en.wikipedia.org//w/index.php?title=Na...,"[Great_Comet_of_1811, Dave_Malloy, War_and_Pea...","Natasha,_Pierre_%26_The_Great_Comet_of_1812",192,True,43
1826,Sarbanes–Oxley Act,https://en.wikipedia.org//w/index.php?title=Sa...,"[107th_United_States_Congress, United_States_S...",Sarbanes%E2%80%93Oxley_Act,253,True,38
1827,Divergence,https://en.wikipedia.org//w/index.php?title=Di...,"[Divergent_series, Divergence_(statistics), Di...",Divergence,178,True,21
1828,Cardiac output,https://en.wikipedia.org//w/index.php?title=Ca...,"[Cardiac_physiology, Heart, Ventricle_(heart),...",Cardiac_output,324,True,28


In [1]:
import csv
import ast
from neo4j import GraphDatabase

In [2]:
# --- Configurações da Conexão ---
URI = "bolt://neo.andersonlbsoares.com.br:7687"
AUTH = ("neo4j", "your_password")
DATABASE = "anderson"

# --- Configurações do Processamento ---
CHUNK_SIZE = 2000

In [ ]:
def create_uniqueness_constraint(driver):
    """Garante que a propriedade 'title_encode' seja única para os nós do tipo Document."""
    with driver.session(database=DATABASE) as session:
        try:
            session.run("""
                CREATE CONSTRAINT IF NOT EXISTS FOR (d:Document)
                REQUIRE d.title_encode IS UNIQUE
            """)
            print("Restrição de unicidade para 'title_encode' criada.")
        except Exception as e:
            print(f"Erro ao criar a restrição de unicidade: {e}")


def create_nodes_in_batch(driver, file_path, batch_size):
    """Lê o arquivo CSV e cria os nós no Neo4J em lotes."""
    print("Iniciando a criação de nós...")
    with open(file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        batch = []
        count = 0
        for row in reader:
            # Adiciona apenas as propriedades necessárias para o nó
            node_properties = {
                'document_title': row['document_title'],
                'document_url': row['document_url'],
                'title_encode': row['title_encode']
            }
            batch.append(node_properties)

            if len(batch) >= batch_size:
                _execute_node_batch(driver, batch)
                count += len(batch)
                print(f"{count} nós processados...")
                batch = []

        # Processa o último lote, se houver
        if batch:
            _execute_node_batch(driver, batch)
            count += len(batch)
            print(f"{count} nós processados...")

    print("Criação de nós concluída.")

def _execute_node_batch(driver, batch):
    """Executa a query para criar um lote de nós."""
    with driver.session(database=DATABASE) as session:
        # A query UNWIND + MERGE é a forma mais eficiente de criar nós em lote.
        # MERGE garante que, se um nó com o mesmo 'title_encode' já existir, ele não será duplicado.
        session.run("""
            UNWIND $batch as row
            MERGE (d:Document {title_encode: row.title_encode})
            ON CREATE SET
                d.document_title = row.document_title,
                d.document_url = row.document_url
        """, batch=batch)


def create_relationships_in_batch(driver, file_path, batch_size):
    """Lê o arquivo CSV e cria as relações 'CITES' em lotes."""
    print("\nIniciando a criação de relacionamentos...")
    with open(file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        batch = []
        count = 0
        for row in reader:
            source_title_encode = row['title_encode']
            try:
                # A coluna 'refs' parece ser uma string de uma lista Python
                target_title_encodes = ast.literal_eval(row['refs'])
                if isinstance(target_title_encodes, list):
                    batch.append({
                        'source': source_title_encode,
                        'targets': target_title_encodes
                    })
            except (ValueError, SyntaxError):
                # Ignora a linha se o 'refs' for malformado
                print(f"Aviso: Não foi possível processar a coluna 'refs' para o documento '{source_title_encode}'. Valor: {row['refs']}")
                continue

            if len(batch) >= batch_size:
                _execute_relationship_batch(driver, batch)
                count += 1
                print(f"Lote {count} de relacionamentos processado...")
                batch = []

        # Processa o último lote, se houver
        if batch:
            _execute_relationship_batch(driver, batch)
            count += 1
            print(f"Lote {count} de relacionamentos processado...")

    print("Criação de relacionamentos concluída.")

def _execute_relationship_batch(driver, batch):
    """Executa a query para criar um lote de relacionamentos."""
    with driver.session(database=DATABASE) as session:
        # Esta query é otimizada para criar relações em lote.
        # Ela encontra o nó de origem e todos os nós de destino (que já devem existir)
        # e cria a relação 'CITES' entre eles.
        session.run("""
            UNWIND $batch as row
            MATCH (source:Document {title_encode: row.source})
            UNWIND row.targets as target_title
            MATCH (target:Document {title_encode: target_title})
            WHERE target IS NOT NULL 
            MERGE (source)-[:CITES]->(target)
        """, batch=batch)

Rodei depois para corrigir os 1641 nós extras criados

driver = GraphDatabase.driver("bolt://neo.andersonlbsoares.com.br:7687", auth=("neo4j", "your_password"))

with driver.session(database="anderson") as session:
    # Pega todos os titles do Neo4j
    results = session.run("MATCH (n) RETURN n.title_encode AS title")
    titulos_neo4j = set([record["title"] for record in results if record["title"] is not None])

    # Encontra títulos extras
extras = titulos_neo4j - titulos_csv
print(f"Nós extras: {len(extras)}")
print(extras)

extras_list = list(extras)
 with driver.session(database="anderson") as session:
     session.run("""
         MATCH (n:Document)
         WHERE n.title_encode IN $extras
         DETACH DELETE n
     """, extras=extras_list)

In [5]:
try:
    csv.field_size_limit(10 * 1024 * 1024)

    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        print("Conexão com Neo4J estabelecida com sucesso.")

        # 1. Garante que a restrição de unicidade exista (uma vez só)
        create_uniqueness_constraint(driver)

        # 2. Itera pelos 50 arquivos CSV
        for idx in range(50):
            CSV_FILE_PATH = f"csv/nq-train-{idx:02d}.csv"
            print(f"\nProcessando arquivo: {CSV_FILE_PATH}")

            # Cria todos os nós em lotes
            create_nodes_in_batch(driver, CSV_FILE_PATH, CHUNK_SIZE)

            # Cria todos os relacionamentos em lotes
            create_relationships_in_batch(driver, CSV_FILE_PATH, CHUNK_SIZE)

        print("\nProcesso de população do grafo concluído!")

except Exception as e:
    print(f"Ocorreu um erro: {e}")

Conexão com Neo4J estabelecida com sucesso.
Restrição de unicidade para 'title_encode' garantida.

Processando arquivo: csv/nq-train-00.csv
Iniciando a criação de nós...
2000 nós processados...
4000 nós processados...
5961 nós processados...
Criação de nós concluída.

Iniciando a criação de relacionamentos...
Lote 1 de relacionamentos processado...
Lote 2 de relacionamentos processado...
Lote 3 de relacionamentos processado...
Criação de relacionamentos concluída.

Processando arquivo: csv/nq-train-01.csv
Iniciando a criação de nós...
2000 nós processados...
4000 nós processados...
6000 nós processados...
6138 nós processados...
Criação de nós concluída.

Iniciando a criação de relacionamentos...
Lote 1 de relacionamentos processado...
Lote 2 de relacionamentos processado...
Lote 3 de relacionamentos processado...
Lote 4 de relacionamentos processado...
Criação de relacionamentos concluída.

Processando arquivo: csv/nq-train-02.csv
Iniciando a criação de nós...
2000 nós processados...
